In [ ]:
pip install transformers==3.0.2 sentence_transformers==0.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 14.0 MB/s 
     |████████████████████████████████| 65 kB 4.5 MB/s 
     |████████████████████████████████| 880 kB 56.8 MB/s 
     |████████████████████████████████| 3.0 MB 46.7 MB/s 
     |████████████████████████████████| 1.3 MB 50.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.3-py3-none-any.whl size=97302 sha256=d9c3283cca8d15b166aa6d6dc7d4e6fe9c63c7a40b36ad19289c9cfa4b24d6de
  Stored in directory: /root/.cache/pip/wheels/9e/03/96/a32af02b51743e0f3812c48cfc6d37ee8a6eaf6706b6014876
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=41f2423450c54aba544226b22b4976fe3fc65deae0abff4211586aae58dd9a46
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sacremoses


In [ ]:
import re
import os
import json
import pickle
import numpy as np
import pandas as pd
import random
import torch 
from torch import nn
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Assignments/capstone/phrase-bert-topic-model-master/phrase-topic-model/

/content/drive/MyDrive/Assignments/capstone/phrase-bert-topic-model-master/phrase-topic-model


In [ ]:
from model.dae_model import DictionaryAutoencoder
from model_utils import run_epoch, text_to_topic, rank_topics_by_percentage

In [ ]:
cd /content/drive/MyDrive/Assignments/capstone/

/content/drive/MyDrive/Assignments/capstone


In [ ]:
ls

 2017_headline_results.csv         pntm_headlines.ipynb
 FINAL_semeval2010t8_train.csv     pntm_semeval.ipynb
 headline_topic_df_100.csv         pntm_semeval_labels.ipynb
 phrase_bert_similarity.ipynb     'pooled_context_para_triples_p=0.8'/
 phrase-bert-topic-model-master/   semeval2010t8_test.csv
 pntm/                             semeval2010t8_train.csv


In [ ]:
headline = pd.read_csv('2017_headline_results.csv')

In [ ]:
headline.head(5)

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,polusa,/home/vv2116/capstone_data/2017_1.csv,115892789,0,115892789,obama boosted white house technology; trump se...,obama boosted white house technology; <ARG1> t...,1,0,NaN,2
1,polusa,/home/vv2116/capstone_data/2017_1.csv,4501005,0,4501005,gun-shaped cell phone case prompts police stan...,<ARG0> gun-shaped cell phone case </ARG0> prom...,1,1,NaN,2
2,polusa,/home/vv2116/capstone_data/2017_1.csv,59608821,0,59608821,23 dead after ferry carrying new year's revele...,"<ARG1> 23 dead after ferry carrying new ""year'...",1,1,NaN,2
3,polusa,/home/vv2116/capstone_data/2017_1.csv,59542836,0,59542836,car chase leads to 2017's first officer-involv...,"<ARG0> car chase </ARG0> leads to <ARG1> ""2017...",1,1,NaN,2
4,polusa,/home/vv2116/capstone_data/2017_1.csv,52947165,0,52947165,cybersecurity expert on russian sanctions with...,<ARG1> cybersecurity expert </ARG1> on russian...,1,0,NaN,2


In [ ]:
# filter the dataframe by pair_label for only causal texts
headline_causal = headline[headline['pair_label'] == 1] 
len(headline_causal), len(headline)

(14519, 21752)

In [ ]:
headline_causal.head(5)

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
1,polusa,/home/vv2116/capstone_data/2017_1.csv,4501005,0,4501005,gun-shaped cell phone case prompts police stan...,<ARG0> gun-shaped cell phone case </ARG0> prom...,1,1,NaN,2
2,polusa,/home/vv2116/capstone_data/2017_1.csv,59608821,0,59608821,23 dead after ferry carrying new year's revele...,"<ARG1> 23 dead after ferry carrying new ""year'...",1,1,NaN,2
3,polusa,/home/vv2116/capstone_data/2017_1.csv,59542836,0,59542836,car chase leads to 2017's first officer-involv...,"<ARG0> car chase </ARG0> leads to <ARG1> ""2017...",1,1,NaN,2
9,polusa,/home/vv2116/capstone_data/2017_1.csv,55222384,0,55222384,scenes of chaos and mourning after istanbul ni...,<ARG1> scenes of chaos and </ARG1> mourning af...,1,1,NaN,2
11,polusa,/home/vv2116/capstone_data/2017_1.csv,115912220,0,115912220,officials: delaware man injured in istanbul ni...,officials: delaware man injured in istanbul ni...,1,1,NaN,2


In [ ]:
# textwpairs column
headline_textwpairs = headline_causal['text_w_pairs']

In [ ]:
headline_textwpairs.head(5)

1     <ARG0> gun-shaped cell phone case </ARG0> prom...
2     <ARG1> 23 dead after ferry carrying new "year'...
3     <ARG0> car chase </ARG0> leads to <ARG1> "2017...
9     <ARG1> scenes of chaos and </ARG1> mourning af...
11    officials: delaware man injured in istanbul ni...
Name: text_w_pairs, dtype: object

In [ ]:
def extract_args(dataset):
    arg0s = []
    arg1s = []
    for textwpair in dataset:
        arg0 = re.findall(r"<ARG0>(.*?)</ARG0>", textwpair) # list of all argument0s in string textwpair
        arg1 = re.findall(r"<ARG1>(.*?)</ARG1>", textwpair) # list of all argument1s in string textwpair
        if len(arg0) != 0:
            # unpack the list of argument0s and append them one by one
            for arg in arg0:
                arg0s.append(arg)
        if len(arg1) != 0:
            # unpack the list of argument1s and append them one by one
            for arg in arg1:
                arg1s.append(arg)
    return arg0s, arg1s

In [ ]:
# list of arg0s and arg1s for POLUSA headlines
headline_arg0s, headline_arg1s = extract_args(headline_textwpairs)
# list of all args for POLUSA headlines
headline_args = headline_arg0s + headline_arg1s

In [ ]:
# construct text_list
headline_text = headline_causal['text']
headline_text_list = headline_text.tolist()

In [ ]:
# construct dictionaries of word2id
headline_word2id = {val : idx for idx, val in enumerate(set(headline_args))}
len(headline_word2id.keys()), len(headline_args) # duplicates

(20836, 23785)

In [ ]:
# construct dictionaries of id2word
headline_id2word = {val: key for key, val in headline_word2id.items()}

In [ ]:
# contruct dictionaries of id2freq
headline_id2freq = headline_id2word.copy()
headline_freq = [(arg, headline_args.count(arg)) for arg in set(headline_args)]
i = 0
for key, val in headline_id2freq.items():
    headline_id2freq[key] = headline_freq[i][1]
    i += 1

In [ ]:
# load the Phrase-BERT model through the sentence-BERT interface
model_path = "/content/drive/MyDrive/Assignments/capstone/pooled_context_para_triples_p=0.8/"
model = SentenceTransformer(model_path)

In [ ]:
"""
# commented out because results are already saved and can be loaded from files
# compute phrase embeddings using Phrase-BERT
headline_phrase_embs = model.encode(set(headline_args), batch_size=8, show_progress_bar=True)
headline_embs = np.asarray(headline_phrase_embs)
"""

'\n# commented out because results are already saved and can be loaded from files\n# compute phrase embeddings using Phrase-BERT\nheadline_phrase_embs = model.encode(set(headline_args), batch_size=8, show_progress_bar=True)\nheadline_embs = np.asarray(headline_phrase_embs)\n'

In [ ]:
"""
# save the results
topic_model_data_path = "/content/drive/MyDrive/Assignments/capstone/pntm/"
np.save(os.path.join(topic_model_data_path, 'headline_embs_matrix_np'), headline_embs)
"""

'\n# save the results\ntopic_model_data_path = "/content/drive/MyDrive/Assignments/capstone/pntm/"\nnp.save(os.path.join(topic_model_data_path, \'headline_embs_matrix_np\'), headline_embs)\n'

In [ ]:
# set seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
topic_model_data_path = "/content/drive/MyDrive/Assignments/capstone/pntm/"

In [ ]:
headline_embs_matrix_np = np.load(os.path.join(topic_model_data_path, f"headline_embs_matrix_np.npy"))
print(f"Loaded headline word embedding from {topic_model_data_path}")
print(f"Loaded vocab size of {len(headline_word2id)} (including phrases)")

Loaded headline word embedding from /content/drive/MyDrive/Assignments/capstone/pntm/
Loaded vocab size of 20836 (including phrases)


In [ ]:
len(headline_embs_matrix_np)

20836

Below code is adapted from Phrase-Bert: https://github.com/sf-wa-326/phrase-bert-topic-model

In [ ]:
# word frequency and filter info

# compute the length (in n-grams)
# setting word_threshould really high to include every phrase, reset to lower value to remove longer phrases
word_threshold = 100

headline_len_words = [0] * len(headline_id2word)
for (id, word) in headline_id2word.items():
    headline_len_words[id] = len(word.split(' '))
# setting word_len to 10 because 
# and args contain empty spaces at the beginning and end of the strings
headline_indices_to_remove_based_on_len = [id 
                                           for id, word_len 
                                           in enumerate(headline_len_words) 
                                           if (word_len > word_threshold )]

print(len(headline_indices_to_remove_based_on_len)) # 0

0


In [ ]:
# keeping every token / word, reset freq_threshold to remove lower frequency words
freq_threshold = 0

headline_sorted_ids = [k for k, v in sorted(headline_id2freq.items(), key=lambda item: item[1])]
headline_sorted_ids.reverse()
headline_indices_to_remove_based_on_freq = [k for k, v in headline_id2freq.items() if v <= freq_threshold ]
headline_to_be_removed = list(set(headline_indices_to_remove_based_on_freq + headline_indices_to_remove_based_on_len))

In [ ]:
# encode the text_lists
headline_text_rep_list = model.encode(headline_text, batch_size = 8, show_progress_bar = True)

Batches:   0%|          | 0/1815 [00:00<?, ?it/s]

In [ ]:
emb_model = "phrase-bert"
print(f"Building sentence model by using {emb_model} as embedding model")

headline_uid_input_vector_list = [(i, headline_text_rep_list[i]) for i in range(len(headline_text_rep_list))]
print(f"Computed {len(headline_uid_input_vector_list)} positive examples")

Building sentence model by using phrase-bert as embedding model
Computed 14519 positive examples


In [ ]:
# setting the argument num_negative_samples for negative sampling
num_neg_samples = 5 # default in the original model

headline_uid_input_vector_list_neg = []
indices = list(range(len(headline_uid_input_vector_list)))
for idx in range(len(headline_uid_input_vector_list)):
    indices_candidate = indices
    neg_indices = random.sample(indices_candidate, num_neg_samples)
    neg_samples = [headline_uid_input_vector_list[neg_i][1] for neg_i in neg_indices]
    neg_vector = np.mean(neg_samples, axis=0)
    headline_uid_input_vector_list_neg.append(neg_vector)
print(f"Computed {len(headline_uid_input_vector_list_neg)} negative examples")

Computed 14519 negative examples


In [ ]:
# set up hyperparameters
headline_net_params = {}
headline_net_params["mode"] = "bert"
headline_net_params["embedding"] = headline_embs_matrix_np
headline_net_params["d_hid"] = 100
headline_net_params["num_rows"] = 100  # number of topics
headline_net_params["num_sub_topics"] = 0
headline_net_params["word_dropout_prob"] = 0.2
headline_net_params["vrev"] = headline_id2word  # idx to word map
headline_net_params["device"] = 'cuda'
headline_net_params["pred_world"] = False

In [ ]:
headline_net = DictionaryAutoencoder(net_params=headline_net_params)
headline_net.to('cuda')

DictionaryAutoencoder(
  (embeddings): Embedding(20836, 768)
  (W_proj): Linear(in_features=768, out_features=100, bias=True)
  (act): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (W_att): Linear(in_features=100, out_features=768, bias=True)
  (W_out): Linear(in_features=768, out_features=20836, bias=True)
)

In [ ]:
# training specs (default from original code)
num_epochs = 300
batch_size = 100
ortho_weight = 1e-5
world_clas_weight = 0.0
headline_optim = torch.optim.Adam(headline_net.parameters(), lr=1e-4)
interpret_interval = int(np.ceil(num_epochs / 10))
h_model = 2

In [ ]:
# iterating through batches
headline_batch_intervals = [
    (start, start + batch_size)
    for start in range(0, len(headline_uid_input_vector_list), batch_size)]
    # batch_intervals = batch_intervals[:100]
headline_split = int(np.ceil(len(headline_batch_intervals) * 0.9))
headline_batch_intervals_train = headline_batch_intervals[:headline_split]
headline_batch_intervals_valid = headline_batch_intervals[headline_split:]

In [ ]:
import argparse
parser = argparse.ArgumentParser()
"""parser.add_argument("--lr", type=float, default=1e-4)
"""
args = parser.parse_args(args=[])
args.device = 'cuda:' + '0'
args.triplet_loss_margin = 1.0
args.triplet_loss_weight = 1.0
args.ortho_weight = 1e-5
args.neighbour_loss_weight = 1e-7
args.offset_loss_weight = 1e-4

In [ ]:
# headline training
print("\n" + "=" * 70)
for epoch in range(num_epochs):
    # training
    headline_net.train()
    train_mode = True
    print(f"Epoch {epoch}")
    run_epoch(headline_net, headline_optim, headline_batch_intervals_train,
              headline_uid_input_vector_list, headline_uid_input_vector_list_neg,
              args, train_mode, h_model, epoch, 200)

    # validation
    headline_net.eval()
    train_mode = False
    with torch.no_grad():
        run_epoch(
                headline_net,
                headline_optim,
                headline_batch_intervals_valid,
                headline_uid_input_vector_list,
                headline_uid_input_vector_list_neg,
                args, 
                train_mode,
                h_model,
                epoch,
                200
        )

    if (epoch + 1) % interpret_interval == 0:
        print("Topics with probability argmax")
        topics_print_list = headline_net.rank_vocab_for_topics(
            word_embedding_matrix=headline_embs_matrix_np,
            to_be_removed=headline_to_be_removed)
        print("=" * 70)

    print()
    print()
    print()
    print("=" * 70)


Epoch 0
[TRAIN] loss: 0.8197, 0.8197, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.89 s
[VALID] loss: 0.7443, 0.7443, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.03 s



Epoch 1
[TRAIN] loss: 0.7038, 0.7038, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.64 s
[VALID] loss: 0.6691, 0.6691, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.03 s



Epoch 2
[TRAIN] loss: 0.6353, 0.6353, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.61 s
[VALID] loss: 0.6156, 0.6156, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.03 s



Epoch 3
[TRAIN] loss: 0.5924, 0.5924, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.60 s
[VALID] loss: 0.5849, 0.5849, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.03 s



Epoch 4
[TRAIN] loss: 0.5658, 0.5658, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.60 s
[VALID] loss: 0.5628, 0.5628, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.03 s



Epoch 5
[TRAIN] loss: 0.

In [ ]:
# headline print topic list
print("Finally after training")
headline_net.eval()

print("Topics with probability argmax")
prob_over_vocab_np, topics_print_list = headline_net.rank_vocab_for_topics(
            word_embedding_matrix=headline_embs_matrix_np, to_be_removed=headline_to_be_removed
    )
print("=" * 70)

Finally after training
Topics with probability argmax
100
100
[ 8807 14781 19059 20468 11307 10974  9522 15684  6621  5691]
topic 0 :  "'going" ,  change in school curriculum ,  precious-gold up on weaker dollar, ,  pension obligations crush brazil, ,  fears for immigrant communities across us ,  plane collides with truck at lax, ,  evacuations ordered ,  america is a home for the "world's" most talented. ,  buoyant german ,  british "consumers'" confidence slumps 
[12130 18595 18016 15865 10109 20487 10341 13976 14920  1112]
topic 1 :  students selecting a college increasingly ,  the uk economy breaks more records ,  how students are turning schools ,  plug gaps in asylum system ,  u.s. travel ban start looms ,  microbiome science could ,  holding independence vote ,  "'overworked" and "underpaid'" nhs staff ,  islamic state loses caliphate ,  nepal quake survivors rebuild 
[13668  3337 11382   424 11205 15300 13002  5847  3780  6251]
topic 2 :  strike on syria "'act" of "aggression',

In [ ]:
# after training we evaluate all the topics percentage in the dataset and rank the topics by percentage
uid_list, vector_list = zip(*headline_uid_input_vector_list)
topic_pred_list = text_to_topic(vector_list, headline_net, 'cuda')

topic_id_ranked, topic_percentage_ranked = rank_topics_by_percentage( topic_pred_list )

for rank, (topic_id, topic_percentage) in enumerate( zip(topic_id_ranked, topic_percentage_ranked)):
    print(
            f"Rank: {rank}, Topic_id: {topic_id}, Topic Words: {topics_print_list[topic_id]}, \
            Topic Percentage: {topic_percentage}"
        )

100%|██████████| 37/37 [00:00<00:00, 650.88it/s]

Rank: 0, Topic_id: 33, Topic Words: topic 33 :  parents convicted in "son's" death ,  ‘lax immigration enforcement’ ,  targeting bame staff ,  sidelined democrats ,  to close aging nuclear plant ,  directors of new homes warranty firm ,  renaming ,  markets slump ,  and it cannot lie. ,  maybe ,             Topic Percentage: 4.15
Rank: 1, Topic_id: 17, Topic Words: topic 17 :  black entrepreneurs ,  south "korea's" liberal new president ,  that will lead ,  german economy accelerated in 2016 ,  pelosi confident of her "caucus'" support ,  beer sales "don't" take a hit ,  green prawn imports suspended ,  trump ends health payments ,  japan needs more workers ,  trump fired comey over russia, ,             Topic Percentage: 3.89
Rank: 2, Topic_id: 3, Topic Words: topic 3 :  plagued by callers to "'babestation'" sex chatlines ,  to hold south carolina company in contempt ,  warnings of "'timebomb'" ,  child sexual abuse scandals ,  drug company looks to sell ,  to drop debit rule ,  attor

In [ ]:
prob_over_vocab_df = pd.DataFrame(prob_over_vocab_np)
prob_over_vocab_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,20826,20827,20828,20829,20830,20831,20832,20833,20834,20835
0,0.000052,0.000049,0.000044,0.000034,0.000084,0.000071,0.000062,0.000061,0.000046,0.000054,...,0.000086,0.000039,0.000034,0.000093,0.000036,0.000051,0.000045,0.000040,0.000042,0.000032
1,0.000052,0.000054,0.000042,0.000058,0.000032,0.000033,0.000044,0.000047,0.000050,0.000042,...,0.000039,0.000054,0.000037,0.000026,0.000042,0.000026,0.000055,0.000046,0.000056,0.000069
2,0.000046,0.000039,0.000078,0.000034,0.000046,0.000053,0.000039,0.000073,0.000051,0.000056,...,0.000044,0.000039,0.000050,0.000062,0.000041,0.000059,0.000049,0.000059,0.000053,0.000045
3,0.000044,0.000032,0.000044,0.000114,0.000042,0.000097,0.000087,0.000056,0.000037,0.000035,...,0.000051,0.000040,0.000050,0.000038,0.000074,0.000054,0.000043,0.000043,0.000043,0.000033
4,0.000031,0.000041,0.000043,0.000088,0.000040,0.000054,0.000043,0.000037,0.000051,0.000040,...,0.000026,0.000043,0.000031,0.000040,0.000050,0.000036,0.000052,0.000044,0.000058,0.000072


In [ ]:
prob_over_vocab_df.idxmax() # for each column, find the row number of the max

0        62
1        54
2        80
3         3
4        17
         ..
20831    91
20832    93
20833    66
20834    67
20835    19
Length: 20836, dtype: int64

In [ ]:
df = pd.DataFrame(prob_over_vocab_df.idxmax())
df = df.reset_index()
df

,index,0
0,0,62
1,1,54
2,2,80
3,3,3
4,4,17
...,...,...
20831,20831,91
20832,20832,93
20833,20833,66
20834,20834,67


In [ ]:
# construct a dictionary with arg ids as keys and topic ids as values
argid2topic_dict = df[0].to_dict()

In [ ]:
# construct a dictionary with topic ids as keys and arg ids as values
# where only one topic is assigned to each argument
topic2argid_dict = df.groupby(0)['index'].apply(list).to_dict()

In [ ]:
argid2topic_dict

{0: 62,
 1: 54,
 2: 80,
 3: 3,
 4: 17,
 5: 3,
 6: 3,
 7: 9,
 8: 41,
 9: 71,
 10: 97,
 11: 56,
 12: 72,
 13: 19,
 14: 17,
 15: 99,
 16: 97,
 17: 50,
 18: 53,
 19: 45,
 20: 0,
 21: 14,
 22: 31,
 23: 44,
 24: 89,
 25: 98,
 26: 8,
 27: 78,
 28: 72,
 29: 43,
 30: 16,
 31: 52,
 32: 3,
 33: 92,
 34: 61,
 35: 0,
 36: 9,
 37: 10,
 38: 69,
 39: 89,
 40: 32,
 41: 87,
 42: 80,
 43: 59,
 44: 96,
 45: 39,
 46: 94,
 47: 17,
 48: 72,
 49: 3,
 50: 80,
 51: 49,
 52: 98,
 53: 66,
 54: 21,
 55: 60,
 56: 72,
 57: 0,
 58: 31,
 59: 99,
 60: 13,
 61: 84,
 62: 29,
 63: 24,
 64: 6,
 65: 65,
 66: 45,
 67: 4,
 68: 84,
 69: 69,
 70: 50,
 71: 53,
 72: 89,
 73: 51,
 74: 97,
 75: 25,
 76: 9,
 77: 25,
 78: 3,
 79: 22,
 80: 65,
 81: 14,
 82: 98,
 83: 46,
 84: 4,
 85: 3,
 86: 58,
 87: 50,
 88: 99,
 89: 76,
 90: 50,
 91: 26,
 92: 24,
 93: 43,
 94: 73,
 95: 61,
 96: 83,
 97: 39,
 98: 30,
 99: 5,
 100: 29,
 101: 52,
 102: 21,
 103: 25,
 104: 66,
 105: 71,
 106: 76,
 107: 18,
 108: 77,
 109: 80,
 110: 17,
 111: 57,
 112: 17

In [ ]:
topic2argid_dict[0]

[20,
 35,
 57,
 289,
 422,
 820,
 1041,
 1287,
 1359,
 1705,
 1899,
 2045,
 2100,
 2144,
 2150,
 2696,
 3133,
 3237,
 3277,
 3368,
 3451,
 3551,
 3667,
 3814,
 3832,
 4310,
 4540,
 4693,
 4709,
 5020,
 5129,
 5130,
 5437,
 5669,
 5686,
 5702,
 5751,
 5852,
 5904,
 5923,
 6433,
 6650,
 6663,
 6737,
 6851,
 6983,
 7440,
 7471,
 7486,
 8062,
 8158,
 8386,
 8613,
 8643,
 8801,
 9163,
 9283,
 9305,
 9404,
 9522,
 9558,
 9808,
 10299,
 10362,
 10482,
 10974,
 11042,
 11107,
 11307,
 11709,
 11878,
 12117,
 12597,
 12851,
 12868,
 13013,
 13458,
 13561,
 13729,
 14145,
 14189,
 14238,
 14451,
 15274,
 15382,
 15467,
 15600,
 15684,
 15735,
 16026,
 16126,
 16524,
 16673,
 16946,
 17585,
 17750,
 17843,
 17931,
 18289,
 18496,
 19114,
 19295,
 19346,
 19764,
 19832,
 19858,
 20292,
 20715]

Construct a new dataframe with columns: sentence, arg0, arg1, arg0id, arg1id, arg0topicid, arg1topicid

In [ ]:
headline_topic_df = headline_causal.copy()
headline_topic_df.head(5)

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
1,polusa,/home/vv2116/capstone_data/2017_1.csv,4501005,0,4501005,gun-shaped cell phone case prompts police stan...,<ARG0> gun-shaped cell phone case </ARG0> prom...,1,1,NaN,2
2,polusa,/home/vv2116/capstone_data/2017_1.csv,59608821,0,59608821,23 dead after ferry carrying new year's revele...,"<ARG1> 23 dead after ferry carrying new ""year'...",1,1,NaN,2
3,polusa,/home/vv2116/capstone_data/2017_1.csv,59542836,0,59542836,car chase leads to 2017's first officer-involv...,"<ARG0> car chase </ARG0> leads to <ARG1> ""2017...",1,1,NaN,2
9,polusa,/home/vv2116/capstone_data/2017_1.csv,55222384,0,55222384,scenes of chaos and mourning after istanbul ni...,<ARG1> scenes of chaos and </ARG1> mourning af...,1,1,NaN,2
11,polusa,/home/vv2116/capstone_data/2017_1.csv,115912220,0,115912220,officials: delaware man injured in istanbul ni...,officials: delaware man injured in istanbul ni...,1,1,NaN,2


In [ ]:
# modify the original extract_args function to differentiate which text arguments belong to
def extract_args2(dataset):
    arg0s = []
    arg1s = []
    for textwpair in dataset:
        arg0 = re.findall(r"<ARG0>(.*?)</ARG0>", textwpair) # list of all argument0s in string textwpair
        arg1 = re.findall(r"<ARG1>(.*?)</ARG1>", textwpair) # list of all argument1s in string textwpair
        arg0s.append(arg0)
        arg1s.append(arg1)
    return arg0s, arg1s

In [ ]:
# create new columns for lists of arg0s and arg1s for each text
headline_topic_df['arg0'], headline_topic_df['arg1'] = extract_args2(headline_textwpairs)

In [ ]:
# create arg0_id and arg1_id columns
headline_arg0id_list = []
for arg0s in headline_topic_df['arg0']:
    temp = []
    for arg0 in arg0s:
        temp.append(headline_word2id[arg0])
    headline_arg0id_list.append(temp)
headline_topic_df['arg0_id'] = headline_arg0id_list

headline_arg1id_list = []
for arg1s in headline_topic_df['arg1']:
    temp = []
    for arg1 in arg1s:
        temp.append(headline_word2id[arg1])
    headline_arg1id_list.append(temp)
headline_topic_df['arg1_id'] = headline_arg1id_list

In [ ]:
# create columns for arg0_topicid and arg1_topicid
headline_arg0topic_list = []
for arg0s in headline_arg0id_list:
    temp = []
    for arg0 in arg0s:
        temp.append(argid2topic_dict[arg0])
    headline_arg0topic_list.append(temp)
headline_topic_df['arg0_topicid'] = headline_arg0topic_list

headline_arg1topic_list = []
for arg1s in headline_arg1id_list:
    temp = []
    for arg1 in arg1s:
        temp.append(argid2topic_dict[arg1])
    headline_arg1topic_list.append(temp)
headline_topic_df['arg1_topicid'] = headline_arg1topic_list

In [ ]:
headline_topic_df

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents,arg0,arg1,arg0_id,arg1_id,arg0_topicid,arg1_topicid
1,polusa,/home/vv2116/capstone_data/2017_1.csv,4501005,0,4501005,gun-shaped cell phone case prompts police stan...,<ARG0> gun-shaped cell phone case </ARG0> prom...,1,1,NaN,2,[ gun-shaped cell phone case ],[ police standoff ],[19666],[10359],[62],[22]
2,polusa,/home/vv2116/capstone_data/2017_1.csv,59608821,0,59608821,23 dead after ferry carrying new year's revele...,"<ARG1> 23 dead after ferry carrying new ""year'...",1,1,NaN,2,[],"[ 23 dead after ferry carrying new ""year's"" re...",[],[18586],[],[56]
3,polusa,/home/vv2116/capstone_data/2017_1.csv,59542836,0,59542836,car chase leads to 2017's first officer-involv...,"<ARG0> car chase </ARG0> leads to <ARG1> ""2017...",1,1,NaN,2,[ car chase ],"[ ""2017's"" first officer-involved shooting ]",[8936],[13728],[42],[18]
9,polusa,/home/vv2116/capstone_data/2017_1.csv,55222384,0,55222384,scenes of chaos and mourning after istanbul ni...,<ARG1> scenes of chaos and </ARG1> mourning af...,1,1,NaN,2,[ istanbul nightclub attack ],[ scenes of chaos and ],[5451],[9142],[33],[72]
11,polusa,/home/vv2116/capstone_data/2017_1.csv,115912220,0,115912220,officials: delaware man injured in istanbul ni...,officials: delaware man injured in istanbul ni...,1,1,NaN,2,[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21744,polusa,/home/vv2116/capstone_data/2017_1.csv,17909966,0,17909966,grenfell fire: council leader quits over fire ...,grenfell fire: <ARG1> council leader quits </A...,1,1,NaN,2,[ fire ],[ council leader quits ],[5930],[18708],[99],[13]
21746,polusa,/home/vv2116/capstone_data/2017_1.csv,3902224,0,3902224,weekend roundup: the digital threats to democr...,weekend roundup: <ARG0> the digital threats to...,1,1,NaN,2,[ the digital threats to democracy are mounting ],[],[15190],[],[22],[]
21747,polusa,/home/vv2116/capstone_data/2017_1.csv,3895951,0,3895951,why this congressional candidate refuses to ch...,why <ARG0> this congressional candidate refuse...,1,1,NaN,2,[ this congressional candidate refuses to chan...,[],[14851],[],[50],[]
21749,polusa,/home/vv2116/capstone_data/2017_1.csv,55425829,0,55425829,microsoft to cut sales jobs next week,<ARG1> microsoft </ARG1> <ARG0> to cut sales j...,1,1,NaN,2,[ to cut sales jobs next week ],[ microsoft ],[13814],[8220],[66],[49]


In [ ]:
"""
# save the dataframe as a csv file
topic_model_data_path = "/content/drive/MyDrive/Assignments/capstone/pntm/"
np.save(os.path.join(topic_model_data_path, 'headline_topic_df_100'), headline_topic_df)
headline_topic_df.to_csv('/content/drive/MyDrive/Assignments/capstone/pntm/headline_topic_df_100.csv')
"""

In [ ]:
"""
with open( os.path.join(topic_model_data_path, 'headline_topic_model_100.pt'), "wb") as f:
    torch.save(headline_net, f)
    print(f"Saved model at { os.path.join(topic_model_data_path) }")
"""

Saved model at /content/drive/MyDrive/Assignments/capstone/pntm/


construct a table to keep track of how many times each topic of cause has caused each topic of effect

In [ ]:
headline_topic_count_df = headline_topic_df.copy()
headline_topic_count_df.tail(5)

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents,arg0,arg1,arg0_id,arg1_id,arg0_topicid,arg1_topicid
21744,polusa,/home/vv2116/capstone_data/2017_1.csv,17909966,0,17909966,grenfell fire: council leader quits over fire ...,grenfell fire: <ARG1> council leader quits </A...,1,1,NaN,2,[ fire ],[ council leader quits ],[5930],[18708],[99],[13]
21746,polusa,/home/vv2116/capstone_data/2017_1.csv,3902224,0,3902224,weekend roundup: the digital threats to democr...,weekend roundup: <ARG0> the digital threats to...,1,1,NaN,2,[ the digital threats to democracy are mounting ],[],[15190],[],[22],[]
21747,polusa,/home/vv2116/capstone_data/2017_1.csv,3895951,0,3895951,why this congressional candidate refuses to ch...,why <ARG0> this congressional candidate refuse...,1,1,NaN,2,[ this congressional candidate refuses to chan...,[],[14851],[],[50],[]
21749,polusa,/home/vv2116/capstone_data/2017_1.csv,55425829,0,55425829,microsoft to cut sales jobs next week,<ARG1> microsoft </ARG1> <ARG0> to cut sales j...,1,1,NaN,2,[ to cut sales jobs next week ],[ microsoft ],[13814],[8220],[66],[49]
21751,polusa,/home/vv2116/capstone_data/2017_1.csv,3046412,0,3046412,"magnitude 6.0 quake hits off ecuador coast, da...",<ARG0> magnitude 6.0 quake hits off ecuador co...,1,1,NaN,2,"[ magnitude 6.0 quake hits off ecuador coast, ]",[ damage limited ],[9503],[14864],[43],[91]


In [ ]:
headline_topic_count_df = headline_topic_count_df[['arg0_topicid', 'arg1_topicid']]
headline_topic_count_df

,arg0_topicid,arg1_topicid
1,[62],[22]
2,[],[56]
3,[42],[18]
9,[33],[72]
11,[],[]
...,...,...
21744,[99],[13]
21746,[22],[]
21747,[50],[]
21749,[66],[49]


In [ ]:
headline_topic_count_df = headline_topic_count_df.explode('arg0_topicid')
headline_topic_count_df = headline_topic_count_df.explode('arg1_topicid')
headline_topic_count_df['index'] = headline_topic_count_df.index
headline_topic_count_df

,arg0_topicid,arg1_topicid,index
1,62,22,1
2,NaN,56,2
3,42,18,3
9,33,72,9
11,NaN,NaN,11
...,...,...,...
21744,99,13,21744
21746,22,NaN,21746
21747,50,NaN,21747
21749,66,49,21749


In [ ]:
headline_topic_count_df = headline_topic_count_df.groupby(['arg0_topicid', 'arg1_topicid'])['index'].count().unstack(fill_value=0)
headline_topic_count_df

arg1_topicid,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
arg0_topicid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,3,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,2,1
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,2,0,1,0,0
3,1,1,0,3,4,2,0,0,2,5,...,2,2,2,0,0,7,2,3,10,2
4,1,0,0,3,3,0,1,1,1,2,...,0,2,0,0,0,1,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,1,0,5,1,0,0,0,2,2,...,0,2,0,1,2,3,2,3,1,4
96,1,1,0,3,3,0,0,0,1,2,...,1,0,0,0,0,1,1,4,1,0
97,2,0,1,9,1,0,0,0,0,4,...,3,6,1,0,2,6,4,9,3,1


In [ ]:
headline_topic_count_df.sum().sum()

9184